In [ ]:
config = {
    "max_seq_length": 512,
    "batch_size": 16,
    "lr": 0.001,
    "epochs": 20,
    "mixup":None,
    "device": "cuda:1",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/mixup_512_biocaster_4cate_20200908/",
    "result_file":"../results/mixup_cnn_20200909.csv",
    "patience": 4,
}

In [ ]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaster2df,split_data
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"


In [ ]:
def train():
    data_df = biocaster2df(data_file)
    train_set,dev_set,test_set = split_data(data_df,config["seed"])
    from beta_nlp.models.mixup_cls import MixupModel
    from beta_nlp.utils.common import save_to_csv
    cls = MixupModel(train_set,config)
    cls.train(train_set,dev_set)
    result = cls.test(test_set)
    result.update(config)
    save_to_csv(result, config["result_file"])

In [ ]:
config["mixup"]= "sentence"
config["model"] = "CNN+sentence"
lr_opt= [1,0.5,0.1,0.05,0.01,0.005,0.001]
max_seq_length_opt = [200,400,512,600,800]
dropout_opt = [0, 0.1,0.2,0.3,0.4,0.5 ]
for lr in lr_opt:
    for max_seq_length in max_seq_length_opt:
        for dropout in dropout_opt:
            for i in range(10):
                config["lr"]=lr
                config["max_seq_length"]=max_seq_length
                config["dropout"] = dropout
                train()